In [37]:
from neo4j import GraphDatabase

# Creating DiamondDAO data for demo
Here I will first show what normally the chainverse interface would do, aka link the DAO and its users to the different types of elements.

In [38]:
database_uri = "bolt://localhost:7687"
username = "neo4j"
password = "diamondDAO"

In [70]:
class DatabaseConnector:
    "This class defines the connector, and helper functions to interact with the chose schema"
    
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()
        
    def run_query(self, query, props=None):
        try:
            with self.driver.session() as session:
                result = list(session.run(query, props=props))
            return result
        except:
            print(query)
            print(props)
            raise

In [71]:
connector = DatabaseConnector(database_uri, username, password)

In [7]:
# Let's create the DAO
def create_dao(connector, data):
    query = """
        MERGE (diamond:DAO {{id: {}, name: "{}", website:"{}", twitter:"{}"}})
        """.format(data["id"], data["name"], data["website"], data["twitter"])
    connector.run_query(query)
    
diamondDAO = {"id":1, "name": "DiamondDAO", "website":"https://www.diamonddao.xyz/", "twitter":"https://mobile.twitter.com/dmnddao"}
create_dao(connector, diamondDAO)

In [8]:
# First matching the different services the DAO runs on
def connect_discourse(connector, dao_id, discourse_name, discourse_id):
    query = """
        MATCH (dao:DAO {{id: {}}}), (discourse:DiscourseCategory {{discourseName: "{}", id: {}}})
        MERGE (discourse) -[edge:isPartOf]-> (dao)
        """.format(dao_id, discourse_name, discourse_id)
    connector.run_query(query)

def connect_DAOHaus(connector, dao_id, daohaus_id):
    query = """
        MATCH (dao:DAO {{id: {}}}), (daohaus:DAOHausDao {{id:{}}})
        MERGE (daohaus) -[edge:isPartOf]-> (dao)
    """.format(dao_id, daohaus_id)
    connector.run_query(query)

In [9]:
connect_discourse(connector, 1, "DAOHausDiscourse", 106)
connect_DAOHaus(connector, 1, 792)
connect_DAOHaus(connector, 1, 122)

In [10]:
# Then matching the members
def create_user(connector, user):
    query = """
        MERGE (user:User {{id: {}, name:"{}", twitter:"{}"}})
    """.format(user["id"], user["name"], user["twitter"])
    connector.run_query(query)

def connect_discourse_user(connector, user_id, discourse_username):
    query = """
        MATCH (user:User {{id: {}}}), (discourseUser:DiscourseUser {{username: "{}"}})
        MERGE (user) -[edge:isUser]-> (discourseUser)
    """.format(user_id, discourse_username)
    connector.run_query(query)

def connect_wallet(connector, user_id, address):
    query = """
        MATCH (user:User {{id: {}}}), (wallet:Wallet {{address: "{}"}})
        MERGE (user) -[edge:isWallet]-> (wallet)
    """.format(user_id, address)
    connector.run_query(query)

In [11]:
data = {
    "Xqua": {"name": "XquaInTheMoon", "discourseUsername": "Xqua", "address": "0xF0552F71fDdd72ae6d0F59EfE3fCcFFc39aF21FF".lower(), "twitter": "@XquaInTheMoon"},
    "Jovian": {"name": "Jovian", "discourseUsername": "steffbrowne", "address": "0x3a8af31d3cfe775172853528167bb2b5760bf7b9", "twitter": "@Jovian"},
    "Amphiboly": {"name": "Amphiboly", "discourseUsername": "amphiboly.eth", "address": "0xc75446a6adaef73269dbdece73536977b2b639e0", "twitter": "@amphiboly"},
    "Lemp": {"name": "Lemp", "discourseUsername": "lemp.eth", "address": "0x97b9958facec9acb7adb2bb72a70172cb5a0ea7c", "twitter": "@lemp"},
    "Keating": {"name": "Keating", "address": "0xb8b8bcb1c9e15a9a2325c90b0803e6d225598245", "twitter": "@keating"},
    "FeralChain": {"name": "FeralChain", "address": "0x6a55ae1b0bbbc7edbf346360ddc99b9224205f03", "twitter": "@feralchain"}
}

In [12]:
id_i = 1
for user in data:
    create_user(connector, {"id": id_i, "name": data[user]["name"], "twitter": data[user]["twitter"]})
    if "address" in data[user]:
        connect_wallet(connector, id_i, data[user]["address"])
    if "discourseUsername" in data[user]:
        connect_discourse_user(connector, id_i, data[user]["discourseUsername"])
    id_i += 1

# Let's make a few query examples
Here's a few example queries. First I will look for the DAO users, and how many are voting, and the share distribution

In [86]:
#Let's grab the number of members of the DAO
query = """
MATCH (dao:DAO {id:1})<-[isPartOf]-(daohaus:DAOHausDao)<-[isMemberOf]-(members:Wallet)
RETURN count(DISTINCT members)"""
result = connector.run_query(query)
print("The DAO has {} members accross all DAOHaus instances".format(result[0].value()))

The DAO has 32 members accross all DAOHaus instances


32

In [84]:
query = "MATCH (users:User) RETURN users"
result = connector.run_query(query)

In [85]:
result

[<Record users=<Node id=657364 labels=frozenset({'User'}) properties={'twitter': '@XquaInTheMoon', 'name': 'XquaInTheMoon', 'id': 1}>>,
 <Record users=<Node id=657365 labels=frozenset({'User'}) properties={'twitter': '@Jovian', 'name': 'Jovian', 'id': 2}>>,
 <Record users=<Node id=657366 labels=frozenset({'User'}) properties={'twitter': '@amphiboly', 'name': 'Amphiboly', 'id': 3}>>,
 <Record users=<Node id=657367 labels=frozenset({'User'}) properties={'twitter': '@lemp', 'name': 'Lemp', 'id': 4}>>,
 <Record users=<Node id=657368 labels=frozenset({'User'}) properties={'twitter': '@keating', 'name': 'Keating', 'id': 5}>>,
 <Record users=<Node id=657369 labels=frozenset({'User'}) properties={'twitter': '@feralchain', 'name': 'FeralChain', 'id': 6}>>]